In [430]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm, tqdm_pandas
from geopy.distance import great_circle
import json
from shapely.geometry import shape, Point

### 1. Reverse geocode latitude and longitude to zip code.

Load zip codes and subset NYC zips.

In [408]:
# alternate zips; not very precise since they overlap a lot...
zips_alt = pd.read_csv('data/zipcode-database.csv')
zips_alt = zips_alt[zips_alt.State == 'NY']

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
zips = pd.read_csv('data/zipcode/zipcode.csv')

In [4]:
# should find better way of doing this that isn't just me hand picking the indexes
zips = zips[zips.state == 'NY']
man_si_bx = zips.loc[3343:3646, :]
queens_bk = zips.loc[3845:3971, :]
rockaways = zips.loc[4030:4037, :]
nyc_zips = pd.concat([man_si_bx, queens_bk, rockaways])

In [55]:
# make a list of data
zip_list = zip(zip(nyc_zips.latitude.values, nyc_zips.longitude.values), nyc_zips.zip.values)

Define zip matching functions using spherical distance.

In [40]:
def get_distance(zip_pair, point_pair):
    """Get distance between two (lat, lon) pairs."""
    return great_circle(zip_pair, point_pair).miles

In [189]:
def get_zip(point, zip_list):
    """Get matching zip code given a (lat, lon) pair. Based on distance."""
    # point = (lat, lon)
    distances = [(get_distance(point, p[0]), p[1]) for p in zip_list]
    return min(distances)[1]

In [198]:
def add_zips_df(df, lat, lon):
    """Add zip codes to df with (lat, lon) given df and names of lat, lon columns. 
    Implemented with tqdm to show progress bar.
    """
    df_pairs = zip(df[lat], df[lon])
    zip_results = [get_zip(coord, zip_list) for coord in tqdm(df_pairs)]
    df.loc[:, 'zipcode2'] = zip_results

Define zip matching functions using straight difference in (lat, lon) numbers.

In [183]:
def get_diff(zip_pair, point_pair):
    return sum((abs(zip_pair[0] - point_pair[0]), abs(zip_pair[1] - point_pair[1])))

In [190]:
def get_zip_diff(point, zip_list):
    distances = [(get_diff(point, p[0]), p[1]) for p in zip_list]
    return min(distances)[1]

In [187]:
def add_zips_df_diff(df, lat, lon):
    """Add zip codes to df with (lat, lon) given df and names of lat, lon columns. 
    Implemented with tqdm to show progress bar.
    """
    df_pairs = zip(df[lat], df[lon])
    zip_results = [get_zip_diff(coord, zip_list) for coord in tqdm(df_pairs)]
    df.loc[:, 'zipcode_diff'] = zip_results

Add zips to perceived safety data.

In [130]:
ss = pd.read_csv('data/streetscore_newyorkcity.csv')

In [156]:
add_zips_df(ss, 'latitude', 'longitude')

Add zips to crime data.

In [158]:
crime = pd.read_csv('data/NYPD_7_Major_Felony_Incidents.csv')

In [159]:
def get_lat(x):
    """Get latitude given tuple pair of (latitude, longitude)."""
    try:
        return x.translate(None, "'(),").split()[0]
    except:
        return x
    
def get_lon(x):
    """Get longitude given tuple pair of (latitude, longitude)."""
    try:
        return x.translate(None, "'(),").split()[1]
    except:
        return x

In [160]:
crime['lat'] = crime['Location 1'].apply(get_lat)
crime['lon'] = crime['Location 1'].apply(get_lon)

In [162]:
add_zips_df(crime, 'lat', 'lon')

### 2. Match zip codes to Census ZCTAs

In [286]:
zcta = pd.read_csv('data/zip_to_zcta_2015.csv')

In [300]:
zcta_dict = dict(zip(zcta.ZIP.values, zcta.ZCTA.values))

In [402]:
def add_zcta_df(df, zipcol):
    df['zcta'] = df[zipcol].apply(lambda x: zcta_dict[x] if x in zcta_dict else '')

In [403]:
add_zcta_df(ss, 'zipcode')
add_zcta_df(crime, 'zipcode')

In [407]:
ss[ss.zcta == '']

,latitude,longitude,q-score,zipcode,q_norm,zcta
499,40.708225,-74.016037,26.606802,10250,6.512086,
500,40.708233,-74.016434,28.841745,10250,6.987605,
539,40.708542,-74.015442,27.902399,10250,6.787744,
540,40.708549,-74.015846,27.531588,10250,6.708849,
541,40.708549,-74.016670,30.003716,10250,7.234833,
542,40.708557,-74.017136,34.731983,10250,8.240847,
579,40.708862,-74.014847,24.284918,10250,6.018068,
580,40.708862,-74.016487,31.274067,10250,7.505121,
581,40.708862,-74.015244,28.199366,10250,6.850929,
582,40.708866,-74.016052,26.783216,10250,6.549620,


In [405]:
crime2011 = crime[crime['Occurrence Year'] == 2011]
crime2014 = crime[crime['Occurrence Year'] == 2014]

Pickle

In [164]:
with open('data/ss_withzip.pkl', 'w') as picklefile:
    pickle.dump(ss, picklefile)
    
with open('data/crime_withzip.pkl', 'w') as picklefile:
    pickle.dump(crime, picklefile)

Load stuff

In [426]:
with open('data/ss_orig_nyc.pkl', 'r') as picklefile:
    ssorignyc = pickle.load(picklefile)

In [429]:
ssorignyc.head()

,City,Error in QS Safer,Error in QS Unique,Error in QS Upperclass,File_Location,Heading,ID,Lat,Lon,Pitch,QS Safer,QS Unique,QS Upperclass,zipcode,fips
2,New York City,0.58,0.80,0.61,/images/id_4026_400_300.jpg,7,4026,40.8259,-73.9249,5,4.96,3.79,5.44,10451,"{u'County': {u'FIPS': u'36005', u'name': u'Bro..."
3,New York City,0.41,0.74,0.61,/images/id_4027_400_300.jpg,335,4027,40.7875,-73.9528,10,6.94,6.66,5.87,10029,"{u'County': {u'FIPS': u'36061', u'name': u'New..."
4,New York City,0.58,0.70,0.60,/images/id_4024_400_300.jpg,356,4024,40.7728,-73.9584,12,6.36,6.29,6.47,10021,"{u'County': {u'FIPS': u'36061', u'name': u'New..."
5,New York City,0.54,0.86,0.63,/images/id_4025_400_300.jpg,7,4025,40.6887,-73.8402,8,4.85,4.67,5.08,11416,"{u'County': {u'FIPS': u'36081', u'name': u'Que..."
6,New York City,0.61,0.82,0.59,/images/id_4022_400_300.jpg,19,4022,40.7743,-73.9096,5,6.70,4.59,5.23,11105,"{u'County': {u'FIPS': u'36081', u'name': u'Que..."


In [428]:
ssorignyc.loc[2, 'fips'] 360050063006

{u'Block': {u'FIPS': u'360050063006000'},
 u'County': {u'FIPS': u'36005', u'name': u'Bronx'},
 u'State': {u'FIPS': u'36', u'code': u'NY', u'name': u'New York'},
 u'executionTime': u'107',
 u'status': u'OK'}

### 3. Get Census block group by latitude and longitude.

In [ ]:
# need to optimize this; remove non-NYC shape data

In [431]:
# load geojson file with shapes
with open('data/tl_2010_36_bg10/blockgroups.json', 'r') as f:
    jsbg = json.load(f)

In [432]:
def get_fips(lat, lon):
    """Return fips code given lat and lon."""
    point = Point(lon, lat)
    for feature in jsbg['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return feature['properties']['GEOID10']

In [433]:
%time print get_fips(40.8259, -73.9249)

360050063006
CPU times: user 4.86 s, sys: 68.1 ms, total: 4.93 s
Wall time: 5.14 s


### Compare actual crime vs. perceived safety scores

In [221]:
# def normalize(score, min_score, max_score):
#     norm = (score - min_score) / (max_score - min_score)
#     return norm

#### Perceived safety scores

Normalize q-scores to 0-10 range.

In [241]:
min_score = ss['q-score'].min()
max_score = ss['q-score'].max()

In [242]:
def normalize(score, min_score = min_score, max_score = max_score, range_start = 0, range_end = 10):
    """Normalize q-scores between a given range. Defaults to (0, 10)."""
    norm = range_start + (score - min_score) * (range_end - range_start) / (max_score - min_score)
    # y = 1 + (x-A)*(10-1)/(B-A)
    return norm

In [243]:
ss['q_norm'] = ss['q-score'].apply(normalize)

Get average q-score by zip code.

In [315]:
ss_by_zcta = ss.groupby('zcta')
score_by_zcta = pd.DataFrame(ss_by_zcta.q_norm.mean())
score_by_zcta.sort_values(by = 'q_norm')

,q_norm
zcta,
11693,4.820071
11697,4.874589
10017,4.907698
11430,5.002597
11692,5.035880
11286,5.075041
10474,5.098147
10170,5.233479
11101,5.237702


In [245]:
ss_by_zip = ss.groupby('zipcode')

In [246]:
score_by_zip = pd.DataFrame(ss_by_zip.q_norm.mean())

In [248]:
score_by_zip.sort_values(by = 'q_norm')

,q_norm
zipcode,
10017,4.568792
11693,4.820071
11697,4.874589
11430,5.002597
11692,5.035880
11286,5.075041
10474,5.098147
10280,5.232264
10170,5.233479


Export for cartodb viz

In [251]:
score_by_zip['country'] = 'United States'

In [253]:
score_by_zip.to_csv('data/ss_by_zip.csv')

#### Actual crime scores

In [388]:
def get_crimes_by_zip(df):
    """Given original crimes df, return df of counts by crime type by zip code."""
    byzip = df.groupby(['zcta', 'Offense'])
    crimesdf = byzip.OBJECTID.count().unstack(level=-1)
    crimesdf.fillna(0, inplace = True)
    crimesdf['total'] = crimesdf.sum(axis = 1)
    crimesdf['zcta'] = crimesdf.index
    return crimesdf

In [362]:
def clean_acs(df, zcta_col, data_col):
    """Given ACS zcta data file, return dictionary with key = zcta and values = data."""
    df['zcta'] = df[zcta_col].apply(lambda x: int(x.split()[1]) if len(x.split()) > 1 else x)
    data_dict = dict(zip(df.zcta.values, df[data_col].values.tolist()))
    return data_dict

In [363]:
def clean_acs_df(df, zcta_col, data_col):
    """Given ACS zcta data file, return df of relevant."""
    df['zcta'] = df[zcta_col].apply(lambda x: int(x.split()[1]) if len(x.split()) > 1 else x)
    keep = data_col + ['zcta']
    return df[keep]

In [ ]:
"""normalizing crime:
- crime rate - crime per 100,000 residents
- great post: http://opendata.stackexchange.com/questions/381/how-to-normalize-the-data-when-mapping-crime-reports
"""

In [389]:
crbyzip2014 = get_crimes_by_zip(crime2014)

In [318]:
pop = pd.read_csv('data/acs-pop-by-zcta.csv')

In [364]:
pop_zcta = clean_acs(pop, 'GEO.display-label', ['HC01_VC03'])

In [391]:
crbyzip2014['population'] = crbyzip2014.zcta.apply(lambda x: int(pop_zcta[x][0]) if x in pop_zcta else np.nan)

In [398]:
crbyzip2014['crime_p_1k'] = (crbyzip2014.total / crbyzip2014.population) * 1000

### Test functions

In [61]:
testlist = zip_list[:5]
testpoint = zip_list[0][0]

In [74]:
tp2 = (40.6608, -73.9515)

In [73]:
get_zip(testpoint, testlist)

10001

In [ ]:
40.8115	-73.9668

In [171]:
get_zip_diff((40.7719, -73.9931), zip_list)

11371

In [62]:
print testlist
print testpoint

[((40.750741999999995, -73.996530000000007), 10001), ((40.717040000000004, -73.986999999999995), 10002), ((40.732509, -73.989350000000002), 10003), ((40.706018999999998, -74.008580000000009), 10005), ((40.707903999999999, -74.013419999999996), 10006)]
(40.750741999999995, -73.996530000000007)


In [194]:
# test some more
with open('data/ss_orig_withzips.pkl', 'r') as picklefile:
    ssorig = pickle.load(picklefile)

In [136]:
%time add_zips_df(ssorig)

CPU times: user 11.1 s, sys: 83.9 ms, total: 11.2 s
Wall time: 11.2 s


In [201]:
ssorig.zipcode2 = ssorig.zipcode2.apply(str)
ssorig.zipcode_diff = ssorig.zipcode_diff.apply(str)

In [202]:
ssorig['zip_error_dist'] = ssorig.zipcode == ssorig.zipcode2
ssorig['zip_error_diff'] = ssorig.zipcode == ssorig.zipcode_diff

In [205]:
# 25% incorrect... 
print ssorig.zip_error_dist.value_counts()[0]/float(ssorig.shape[0])
print ssorig.zip_error_diff.value_counts()[0]/float(ssorig.shape[0])

0.256364712848
0.269390171699


In [195]:
add_zips_df_diff(ssorig, 'Lat', 'Lon')

In [199]:
add_zips_df(ssorig, 'Lat', 'Lon')

In [203]:
ssorig

,City,Error in QS Safer,Error in QS Unique,Error in QS Upperclass,File_Location,Heading,ID,Lat,Lon,Pitch,QS Safer,QS Unique,QS Upperclass,zipcode,fips,zipcode_diff,zipcode2,zip_error_dist,zip_error_diff
1226,New York City,0.47,0.58,0.56,/images/id_825_400_300.jpg,123,825,40.6608,-73.9515,4,4.66,4.67,4.62,11225,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11225,11225,True,True
1227,New York City,0.44,0.73,0.55,/images/id_824_400_300.jpg,194,824,40.7667,-73.9385,7,3.98,4.98,4.98,11106,"{u'County': {u'FIPS': u'36081', u'name': u'Que...",11106,11106,True,True
1228,New York City,0.45,0.68,0.54,/images/id_827_400_300.jpg,184,827,40.8115,-73.9668,2,4.81,4.55,4.28,10027,"{u'County': {u'FIPS': u'36061', u'name': u'New...",10115,10115,False,False
1229,New York City,0.35,0.51,0.45,/images/id_826_400_300.jpg,156,826,40.7719,-73.9931,3,3.03,4.37,4.33,10019,"{u'County': {u'FIPS': u'36061', u'name': u'New...",10069,10069,False,False
1230,New York City,0.49,0.71,0.59,/images/id_821_400_300.jpg,166,821,40.7635,-73.8676,4,5.46,4.28,5.06,11369,"{u'County': {u'FIPS': u'36081', u'name': u'Que...",11369,11369,True,True
1232,New York City,0.54,0.74,0.51,/images/id_823_400_300.jpg,152,823,40.7541,-73.9902,5,5.23,5.41,4.93,10018,"{u'County': {u'FIPS': u'36061', u'name': u'New...",10123,10018,True,False
1233,New York City,0.31,0.65,0.14,/images/id_822_400_300.jpg,200,822,40.7266,-73.9390,4,3.21,4.17,3.22,11222,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11222,11222,True,True
1236,New York City,0.49,0.77,0.58,/images/id_4258_400_300.jpg,349,4258,40.6128,-74.0011,17,4.35,4.18,4.03,11214,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11228,11228,False,False
1238,New York City,0.57,0.86,0.77,/images/id_4312_400_300.jpg,20,4312,40.6438,-73.9668,3,6.86,5.15,7.78,11218,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11218,11218,True,True
1256,New York City,0.44,0.50,0.56,/images/id_797_400_300.jpg,167,797,40.6430,-74.0114,1,6.12,6.45,6.30,11220,"{u'County': {u'FIPS': u'36047', u'name': u'Kin...",11220,11220,True,True


In [425]:
ss.head()

,latitude,longitude,q-score,zipcode,q_norm,zcta
0,40.700909,-74.013504,11.062166,10041,3.204716,10004
1,40.701000,-74.013878,10.818611,10041,3.152896,10004
2,40.701080,-74.012878,12.677955,10041,3.548501,10004
3,40.701187,-74.013268,11.417325,10041,3.280282,10004
4,40.701244,-74.012115,25.199091,10041,6.212573,10004
